In [1]:
#Upgrade pip if necessary
!pip install --upgrade pip

In [10]:
#Reading and displaying data file
file_path = '/Users/femi/Downloads/VISSTUCODE/transplantationdatasample.json' 
import json

try:
    # Open and read the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    print("Data Loaded Successfully!") #confirms data loaded successfully
    print(data)  # Print the data 



Data Loaded Successfully!
[{'Recipient': 'B', 'Donor': ['A', 'AB']}, {'Recipient': 'A', 'Donor': ['B', 'AB']}, {'Recipient': 'B', 'Donor': ['AB']}, {'Recipient': 'A', 'Donor': ['AB']}, {'Recipient': 'A', 'Donor': ['B', 'AB']}, {'Recipient': 'B', 'Donor': ['A', 'AB']}, {'Recipient': 'B', 'Donor': ['A', 'AB']}, {'Recipient': 'O', 'Donor': ['A', 'B', 'AB']}, {'Recipient': 'O', 'Donor': ['A', 'B', 'AB']}, {'Recipient': 'A', 'Donor': ['B', 'AB']}, {'Recipient': 'B', 'Donor': ['AB']}, {'Recipient': 'O', 'Donor': ['B', 'AB']}, {'Recipient': 'B', 'Donor': ['A', 'AB']}, {'Recipient': 'A', 'Donor': ['B', 'AB']}, {'Recipient': 'B', 'Donor': ['AB']}, {'Recipient': 'A', 'Donor': ['B', 'AB']}, {'Recipient': 'B', 'Donor': ['A', 'AB']}, {'Recipient': 'O', 'Donor': ['A', 'B', 'AB']}, {'Recipient': 'O', 'Donor': ['A', 'AB']}, {'Recipient': 'A', 'Donor': ['B']}, {'Recipient': 'O', 'Donor': ['A', 'AB']}, {'Recipient': 'O', 'Donor': ['A', 'AB']}, {'Recipient': 'O', 'Donor': ['A', 'B']}, {'Recipient': 'B', 

In [12]:
# Creating Graph

for idx, pair in enumerate(data): #Iterate through each donor-recipient pair in the dataset
    
    donors = pair["Donor"] if isinstance(pair["Donor"], list) else [pair["Donor"]] #'Donor' and 'Recipient' are treated as lists
    recipients = pair["Recipient"] if isinstance(pair["Recipient"], list) else [pair["Recipient"]]

    # Adding each donor-recipient pair as a node in the graph, storing their attributes
    for donor in donors:
        G.add_node(idx, Donor=donor, Recipient=recipients)

        # Iterate through all other pairs to determine any compatibility-based edges
        for other_idx, other_pair in enumerate(data):
            if idx == other_idx:# Skip if its the same donor pair being compared
                
                continue

            
            other_recipients = (
                other_pair["Recipient"] if isinstance(other_pair["Recipient"], list) else [other_pair["Recipient"]] # 'Recipient' in the other pair is treated as a list if needed
            )

            # Checking the compatibility between the donor and recipients of other pairs
            for other_recipient in other_recipients:
                if other_recipient in donor_recipient_compatibility[donor]:
                    # Using directed edge from the current donor to the compatible recipient to finalize edges and nodes
                    G.add_edge(idx, other_idx)


Graph created with nodes and edges.
No Cycles of Length 3 Found


In [14]:
# Create Gurobi optimization model
model = gp.Model() # define model

# Add binary decision variables for each edge in the graph
edge_vars = model.addVars(G.edges, vtype=GRB.BINARY, name="edge")

# Define the objective function: maximize the total number of matches
model.setObjective(gp.quicksum(edge_vars[edge] for edge in G.edges), GRB.MAXIMIZE)

# Constraint to ensure each node has at most one incoming edge
model.addConstrs(
    gp.quicksum(edge_vars[(src, tgt)] for src in G.predecessors(tgt) if (src, tgt) in G.edges) <= 1
    for tgt in G.nodes
)

# Constraints to balance the number of incoming and outgoing edges for each node
model.addConstrs(
    gp.quicksum(edge_vars[(src, tgt)] for src in G.predecessors(tgt) if (src, tgt) in G.edges) ==
    gp.quicksum(edge_vars[(tgt, src)] for src in G.successors(tgt) if (tgt, src) in G.edges)
    for tgt in G.nodes
)

# Solve the optimization problem
model.optimize()

#Print an optimal solution if one was found
if model.status == GRB.OPTIMAL:
    print("Optimization successful!")
    print("Number of successful matches:", model.objVal)
else:
    print("No optimal solution found.")



Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-15
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1450 rows, 186098 columns and 558294 nonzeros
Model fingerprint: 0x664e4e06
Variable types: 0 continuous, 186098 integer (186098 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 660 rows and 108003 columns
Presolve time: 0.28s
Presolved: 790 rows, 78095 columns, 156190 nonzeros
Variable types: 0 continuous, 78095 integer (78095 binary)

Root relaxation: objective 3.700000e+02, 2023 iterations, 0.14 seconds (0.24 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

In [15]:
# Display the number of successful matches
print("Total Successful Matches:", model.objVal)

# Initialize a dictionary to count matches by blood type
match_counts_by_blood_type = {'A': 0, 'B': 0, 'AB': 0, 'O': 0}

# Check if the optimization was successful
if model.status == GRB.OPTIMAL:
    for edge in edge_vars:
        if edge_vars[edge].X > 0.5:  # If the edge is part of the solution
            donor, recipient = edge
            recipient_blood_type = data[recipient]['Recipient']  # Finds blood type of recipient
            match_counts_by_blood_type[recipient_blood_type] += 1

# Print the match results by blood type
print("Match Counts by Blood Type:")
for blood_type, count in match_counts_by_blood_type.items():
    print(f"Blood Type {blood_type}: {count}")


Total Successful Matches: 370.0
Match Counts by Blood Type:
Blood Type A: 185
Blood Type B: 185
Blood Type AB: 0
Blood Type O: 0


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx


# Generate a layout for the graph
layout_positions = nx.spring_layout(G)  # You can also use other layouts like circular_layout

# Create the plot
plt.figure(figsize=(10, 6))
nx.draw(
    G,
    layout_positions,
    node_size=400,
    node_color='skyblue',
    with_labels=True,  # Display node labels
    arrows=True,
    edge_color='gray'
)

# Add blood type labels for nodes
if any('blood_type' in attr for _, attr in G.nodes(data=True)):
    node_labels = {node: attr.get('blood_type', '') for node, attr in G.nodes(data=True)}
    nx.draw_networkx_labels(G, layout_positions, labels=node_labels, font_size=8)

# Finalize and display the plot
plt.title("Visualization of Kidney Donation Matching Graph", fontsize=14)
plt.tight_layout()
plt.show()
